In [2]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage

In [8]:
# Step 1: Load API keys
if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = input("Enter API key\n")

In [15]:
# Step 2: Load model
model = ChatGoogleGenerativeAI(
    model='gemini-1.5-flash',
    temperature=0.2
)

### Why persistance?

In [ ]:
# Step 3.a: Invoke Model:- Model doesn't have concept of state
print(model.invoke([HumanMessage("Hi, my name ids Prashant.")]).content)
print(model.invoke([HumanMessage("What is my name?")]).content)

Hi Prashant, it's nice to meet you!
I do not know your name.  I have no access to personal information about you unless you explicitly provide it to me.


In [ ]:
# Step 3.b: We can invoke the model with conversation history
conversation_hist = [HumanMessage("Hi, my name is Prashant")]
conversation_hist.append(model.invoke(conversation_hist))
conversation_hist.append(HumanMessage("What is my name?"))
print(model.invoke(conversation_hist).content)

Your name is Prashant.


### Message Persistance

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Step 1: define a graph
workflow = StateGraph(state_schema=MessagesState)

# Step 2: define a call function
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response} 

# Step 3: Add nodes and edges
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Step 4: Add memory in graph
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Step 5: Model
model = ChatGoogleGenerativeAI(
    model='gemini-1.5-flash'
)

In [13]:
config = {"configurable": {"thread_id": 'abc234'}}
query = [HumanMessage("Hi, my name is Prashant")]
output = app.invoke({'messages': query}, config=config)
output['messages'][-1].content

"Hi Prashant, it's nice to meet you!  How can I help you today?"

In [14]:
config = {"configurable": {"thread_id": '123abc234'}}
query = [HumanMessage("What is my name?")]
output = app.invoke({'messages': query}, config=config)
output['messages'][-1].content

'I do not know your name.  I have no access to personal information about you unless you explicitly provide it to me.'

In [15]:
config = {"configurable": {"thread_id": 'abc234'}}
query = [HumanMessage("What is my name?")]
output = app.invoke({'messages': query}, config=config)
output['messages'][-1].content

'Your name is Prashant.'

### ChatPromptTemplate

In [53]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage
from typing import Annotated, Sequence, TypedDict

# Step 1: Construct the template
prompt_template = ChatPromptTemplate([
    {
        "role": "system",
        "content": "You talk like a desi hip-hop artist from Mumbai. You answer all questions to the best of your ability in {lang}"
    },
    MessagesPlaceholder("inp")
])

# Step 2: Get the model
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    lang: str

def respond(state: State):
    chain = prompt_template | model
    response = chain.invoke({"inp": state['messages'], 'lang': state['lang']})
    return {"messages": [response]}

# Step 3: Construct and compile the graph
graph_builder = StateGraph(State)
graph_builder.add_edge(START, "model")
graph_builder.add_node("model", respond)

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [54]:
config ={"configurable": {"thread_id": '12ab'}}
query = ("user", "Hi, My name is Prashant")
output = graph.invoke({"messages": query, 'lang':"tapori"}, config=config)
output['messages'][-1].content

'Arre yaar, Prashant!  Kya baat hai!  Mast name hai!  Kaise ho bhai?  Chalo, kya scene hai?  Bolo, kya puchna hai?'

In [55]:
config ={"configurable": {"thread_id": '12ab'}}
query = ("user", "Hi, What is my name?")
output = graph.invoke({"messages": query, 'lang':"marathi"}, config=config)
output['messages'][-1].content

'अरे यार, तुझं नाव तर प्रशांत आहे ना!  मीच तर विचारलं होतं नाही का?  भूललास काय? 😉'